In [11]:
#Se quiere importar un archivo pickle con un diccionario y transformarlo en un dataframe de pandas

import pandas as pd
import pickle
import re
import numpy as np
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])
#Limpiamos la columna de precio quitando el simbolo de € y quitando el /mes

df_def['Precio'] = df_def['Precio'].str.replace('€','')
df_def['Precio'] = df_def['Precio'].str.replace('/mes','')

#transformamos el el precio a numerico	
df_def['Precio'] = df_def['Precio'].str.replace('.','')

#Eliminamos la fila que Precio que pone por consultar
df_def = df_def[df_def['Precio'] != 'precio a consultar']

#Transformamos la columna de precio a numerico
df_def['Precio'] = pd.to_numeric(df_def['Precio'])

#vemos el tipo de datos de cada columna
df_def.dtypes

df_def.head()

,Precio,Descripcion,Otras_caractericas,Caracteristicas,Eficiencia_energetica
https://www.indomio.es/anuncios/95227781/,1155,Se alquila piso amueblado de 3 habitaciones en...,"[Instalación tv individual, Amueblado]",{'Referencia y fecha del anuncio': '635423 - 2...,{'Emisiones de CO₂': 'No indicado'}
https://www.indomio.es/anuncios/95146207/,1540,PISO ESTUDIANTES PARA ENTRAR A PARTIR DEL 1RO ...,"[Exterior, Amueblado]",{'Referencia y fecha del anuncio': '679 - 21/0...,"{'estado': 'Bueno / Habitable', 'certificación..."
https://www.indomio.es/anuncios/94613177/,2500,Disponible para el alquiler MENSUAL O ANUAL. E...,"[Terraza, Amueblado, Trastero]",{'Referencia y fecha del anuncio': 'I-00KU41-W...,{'certificación energética': 'Pediente de cert...
https://www.indomio.es/anuncios/94500757/,2500,Disponible para el alquiler MENSUAL O ANUAL. E...,"[Terraza, Amueblado, Trastero]",{'Referencia y fecha del anuncio': 'I-00KTO8-W...,"{'Eficiencia energética': 'G368 kWh/m² año', '..."
https://www.indomio.es/anuncios/95223135/,550,** El rincón acogedor ** _Porque no necesito m...,"[Instalación tv individual, Amueblado]",{'Referencia y fecha del anuncio': '342394 - 2...,{'Emisiones de CO₂': 'No indicado'}


Tras llamar al dataset y limpiar la columna del precio se va intentar construir un dataframe con columnas de m^2 número de habitaciones, baños etc.

In [19]:
# Lista para almacenar los resultados
resultados = []

for i in range(0, len(df_def)):
    # Obtén la información sobre las habitaciones de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "dormitorio"
        dormitorios_numero = re.search(r'(\d+) dormitorios', habitaciones_info)
        if dormitorios_numero:
            dormitorios = dormitorios_numero.group(1)
        else:
            dormitorios = None
    else:
        dormitorios = None
    
    # Agregar el resultado a la lista
    resultados.append(dormitorios)

# Agregar la lista como una nueva columna al DataFrame
df_def['Habitaciones'] = resultados

df_def["Habitaciones"] = pd.to_numeric(df_def["Habitaciones"])
df_def["Habitaciones"].dtype
#Quiero imputar los valores NaN de la columna Habitaciones con interpolación lineal
# # Imputar los valores NaN con interpolación lineal
df_def["Habitaciones"] = df_def["Habitaciones"].interpolate(method="linear")
# # Mostrar el DataFrame con la nueva columna
print(df_def["Habitaciones"])


0       3.0
1       4.0
2       2.0
3       2.0
4       2.0
       ... 
1553    3.0
1554    2.0
1555    2.0
1556    3.0
1557    2.0
Name: Habitaciones, Length: 1558, dtype: float64


In [21]:
#Ahora se realiza lo mismo con los baños

resultados = []

for i in range(0, len(df_def)):
    # Obtén la información sobre las habitaciones de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "dormitorio"
        dormitorios_numero = re.search(r'(\d+) baño[s]?', habitaciones_info)
        if dormitorios_numero:
            dormitorios = dormitorios_numero.group(1)
        else:
            dormitorios = None
    else:
        dormitorios = None
    
    # Agregar el resultado a la lista
    resultados.append(dormitorios)

# Agregar la lista como una nueva columna al DataFrame
df_def['Baños'] = resultados
df_def["Baños"] = pd.to_numeric(df_def["Baños"])
df_def["Baños"].dtype
#Quiero imputar los valores NaN de la columna Habitaciones con interpolación lineal
# # Imputar los valores NaN con interpolación lineal
df_def["Baños"] = df_def["Baños"].interpolate(method="linear")
# # Mostrar el DataFrame con la nueva columna
print(df_def["Baños"])

0       1.0
1       1.0
2       1.0
3       2.0
4       1.0
       ... 
1553    1.0
1554    2.0
1555    2.0
1556    1.0
1557    1.0
Name: Baños, Length: 1558, dtype: float64


In [22]:
import re

metros_cuadrados_lista = []
for i in range(0, len(df_def)):
    # Convertir la entrada de la columna "Descripcion" a cadena de texto
    texto = str(df_def["Descripcion"][i])
    
    # Buscar los metros cuadrados usando una expresión regular
    metros_cuadrados_match = re.search(r'(\d+(?:\.\d+)?)\s*(?:m²|m2|metros\s+cuadrados|metros)', texto)
    
    # Verificar si se encontró la información de los metros cuadrados
    if metros_cuadrados_match:
        metros_cuadrados = float(metros_cuadrados_match.group(1))  # Convertir a float en lugar de int
    else:
        metros_cuadrados = None
    
    # Agregar el resultado a la lista
    metros_cuadrados_lista.append(metros_cuadrados)

print(metros_cuadrados_lista)

#Ahora transformamos la lista en una columna del dataframe que sea metros cuadrados
df_def["Metros cuadrados"] = metros_cuadrados_lista

df_def["Metros cuadrados"] = pd.to_numeric(df_def["Metros cuadrados"])
df_def["Metros cuadrados"].dtype
#Quiero imputar los valores NaN de la columna Habitaciones con interpolación lineal
# # Imputar los valores NaN con interpolación lineal
df_def["Metros cuadrados"] = df_def["Metros cuadrados"].interpolate(method="linear")
# # Mostrar el DataFrame con la nueva columna
print(df_def["Metros cuadrados"])


[None, 120.0, None, None, None, None, None, None, None, None, None, 99.0, None, 130.0, None, 161.0, 134.0, None, None, 175.0, None, 170.0, None, 161.0, None, None, None, None, None, 130.0, None, None, 90.0, None, None, 800.0, None, None, None, 90.0, 120.0, None, None, None, 2.0, None, None, 165.0, 2.0, None, None, None, None, None, None, None, None, None, 30.0, 200.0, 200.0, None, 46.0, 75.0, None, None, None, None, None, 120.0, None, None, None, None, None, 85.0, None, None, 13.0, None, 13.0, 105.0, None, None, None, None, 120.0, None, None, 13.0, 13.0, 180.0, 10.0, None, 110.0, 75.0, 13.0, None, 125.0, 70.0, 166.0, None, None, None, 93.0, None, 125.0, 125.0, 160.0, 125.0, 90.0, None, 55.0, None, None, 12.0, 90.0, None, None, 138.0, 81.0, None, None, None, None, 75.0, None, None, 88.0, None, None, None, None, 15.0, None, None, 150.0, 93.0, None, None, None, None, None, None, 40.0, 100.0, None, None, 80.0, None, None, None, 50.0, None, None, None, None, None, None, 79.0, None, 135.0, N

In [22]:
#Ahora se va a visualizar el tipo de contrato que se tiene en cada piso.


#Se crea una lista para almacenar los resultados
resultados = []
for i in range(0, len(df_def)):
    contrato = df_def["Caracteristicas"][i].get("contrato")
    
    if contrato:
        contrato = contrato.lower()
    else:
        contrato = "no especificado"
    
    resultados.append(contrato)

df_def["Contrato"] = resultados




C:\Users\alexn\AppData\Local\Temp\ipykernel_14344\3358648040.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  contrato = df_def["Caracteristicas"][i].get("contrato")


In [23]:

#Se cambia el tipo de datos de las columnas habitaciones, baños a numerico y el de contrato a stirng
# df_def["Habitaciones"] = pd.to_numeric(df_def["Habitaciones"])
# df_def["Baños"] = pd.to_numeric(df_def["Baños"])
# df_def["Contrato"] = df_def["Contrato"].astype(str)


df_def.dtypes

Precio                    int64
Descripcion              object
Otras_caractericas       object
Caracteristicas          object
Eficiencia_energetica    object
Habitaciones             object
Baños                    object
Metros cuadrados         object
Contrato                 object
dtype: object

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

# Descargar los recursos necesarios para NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    if text is None:
        return ''
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)
    # Tokenización
    tokens = nltk.word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

df_def['Descripcion'] = df_def['Descripcion'].apply(preprocess_text)

print(df_def['Descripcion'])

tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(df_def['Descripcion'])
print(X)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


https://www.indomio.es/anuncios/95227781/    alquila piso amueblado habitaciones carrer del...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar partir ro febrero juni...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual bajo dormito...
https://www.indomio.es/anuncios/95223135/    rincón acogedor necesito espacio gustará aquí ...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico m valencia valencia solidinmuebles vende...
https://www.indomio.es/anuncios/94958481/    venden pisos obra nueva camino moncada trata r...
https://www.indomio.es/anuncios/95006805/    vende exclusiva viviendas obra nueva calle arq...
https://www.indomio.es/anuncios/95313313/    vende piso barrio tormoseste luminoso apartame...
https://www.indomio.es/anuncios/95319871/    piso 

In [25]:
# Obtener los 5 pisos más comunes basados en la columna 'Descripcion', "Precio", "Habitaciones", "Baños", "Metros cuadrados" y "Contrato"

# Seleccionar las columnas relevantes
columns = ['Descripcion', 'Precio', 'Habitaciones', 'Baños', 'Metros cuadrados', 'Contrato']
df_filtered = df_def[columns]

#descripcion = "con terraza y vistas al mar"
min_bedrooms = 2
max_price = 1500
min_bathrooms = 1
min_sqft = 120
contrato = "alquiler"

# Filtrar el DataFrame por habitaciones, precio, baño y metros cuadrados
filtered_df = df_filtered[(df_filtered['Habitaciones'] >= min_bedrooms) &
                          (df_filtered['Precio'] <= max_price) &
                          (df_filtered['Baños'] >= min_bathrooms) &
                          (df_filtered['Metros cuadrados'] >= min_sqft) &
                          (df_filtered['Contrato'] == contrato)]
                          #(df_filtered['Descripcion'].str.contains(descripcion))

# Crear un vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_filtered = tfidf_vectorizer.fit_transform(filtered_df['Descripcion'])

# Calcular la similitud de coseno entre las descripciones
similarities = cosine_similarity(X_filtered)

# Calcular la similitud total para cada piso
total_similarities = similarities.sum(axis=1)

# Obtener los índices de los 5 pisos más comunes
top_indices = total_similarities.argsort()[::-1][:5]

# Mostrar los 5 pisos más comunes
print(df_filtered.iloc[top_indices])


TypeError: '>=' not supported between instances of 'str' and 'int'